# Merge the two datasets

_Primary Author: Vikram Vadammani_

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sps
from scipy.stats import zscore
from sklearn.preprocessing import MinMaxScaler

In [2]:
PATH = 'data/'

In [3]:
brfssdf = pd.read_csv(PATH + 'brfss_2022.csv')
#brfssdf = brfssdf.head(1000) # limit the df so we can load it easier

# not limiting the df with head or dropna cus we need to see the whole df to make sure the state merge is working. Also dropna kind of drops all the data.
# This means that there are lots of NaNs in the data
print(f'Total rows in BRFSS: {len(brfssdf)}')
brfssdf.head()

Total rows in BRFSS: 445132


,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENM1,...,_SMOKGRP,_LCSREC,DRNKANY6,DROCDY4_,_RFBING6,_DRNKWK2,_RFDRHV8,_FLSHOT7,_PNEUMO3,_AIDTST4
0,1.0,1.0,2032022,2,3,2022,1100.0,2022000001,2.022000e+09,1.0,...,4.0,NaN,2.0,0.0,1.0,0.0,1.0,1.0,2.0,2.0
1,1.0,1.0,2042022,2,4,2022,1100.0,2022000002,2.022000e+09,1.0,...,4.0,NaN,2.0,0.0,1.0,0.0,1.0,2.0,2.0,2.0
2,1.0,1.0,2022022,2,2,2022,1100.0,2022000003,2.022000e+09,1.0,...,4.0,NaN,2.0,0.0,1.0,0.0,1.0,NaN,NaN,2.0
3,1.0,1.0,2032022,2,3,2022,1100.0,2022000004,2.022000e+09,1.0,...,3.0,2.0,2.0,0.0,1.0,0.0,1.0,9.0,9.0,2.0
4,1.0,1.0,2022022,2,2,2022,1100.0,2022000005,2.022000e+09,1.0,...,4.0,NaN,1.0,10.0,1.0,140.0,1.0,NaN,NaN,2.0


In [4]:
# Filter BRFSS dataset to include only 2022
brfssdf = brfssdf[brfssdf["IYEAR"] == 2022].copy()

# Drop the "IYEAR" column from BRFSS after filtering (optional)
brfssdf.drop(columns=["IYEAR"], inplace=True)

In [5]:
#going to find all of the cols that are in anyway related to depression
#A lot of these collumns aent going into the scoring system but it might be nice to see the data of these things AFTER we use the scoring system

#Descriptions of variables  vvvv

cols = ['MENTHLTH','ADDEPEV3', 'SLEPTIM1','EXERANY2','AVEDRNK3','LCSNUMCG', 'ACEDEPRS','LSATISFY','EMTSUPRT', 'SDHISOLT', 'SDHSTRE1','MARIJAN1','POORHLTH']
vardesc = {'MENTHLTH': 'number of bad mental days', 'ADDEPEV3': 'had a depressive episode?', 'SLEPTIM1':'how much sleep', 'EXERANY2': 'exercise in past 30day?', 'AVEDRNK3': 'avg drink past 30day', 'LCSNUMCG': 'number of cigs past 30day', 'ACEDEPRS': 'live with depressed,ill or suici?', 'LSATISFY': 'satisfaction in life?', 'EMTSUPRT': 'how often do you get emo support?', 'SDHISOLT': 'how often socially isolated feeling ', 'SDHSTRE1': 'how often stressed', 'MARIJAN1': 'past 30day weed ', 'POORHLTH': 'number of poor mental or physical days past 30 da'}

# Keep these columns cus they are supported by the PHQ-9 survey AND they have good data in the BRFSS set
scoringcols = ['_STATE','MENTHLTH','ADDEPEV3']
scoringdf = brfssdf[scoringcols].copy()

# Show missing values
missing_values = scoringdf.isna().sum()
print('Missing Values Before Processing:')
print(missing_values[missing_values > 0])


Missing Values Before Processing:
MENTHLTH    3
ADDEPEV3    7
dtype: int64


In [6]:
#now I want to convert the _STATE column from floats to strings

state_mapping = {
    1: 'AL', 2: 'AK', 4: 'AZ', 5: 'AR', 6: 'CA',
    8: 'CO', 9: 'CT', 10: 'DE', 11: 'DC', 12: 'FL', 13: 'GA',
    15: 'HI', 16: 'ID', 17: 'IL', 18: 'IN', 19: 'IA', 20: 'KS', 21: 'KY', 22: 'LA',
    23: 'ME', 24: 'MD', 25: 'MA', 26: 'MI', 27: 'MN', 28: 'MS', 29: 'MO', 30: 'MT',
    31: 'NE', 32: 'NV', 33: 'NH', 34: 'NJ', 35: 'NM', 36: 'NY', 37: 'NC', 38: 'ND',
    39: 'OH', 40: 'OK', 41: 'OR', 42: 'PA', 44: 'RI', 45: 'SC', 46: 'SD', 47: 'TN',
    48: 'TX', 49: 'UT', 50: 'VT', 51: 'VA', 53: 'WA', 54: 'WV', 55: 'WI', 56: 'WY',
    66: 'GU', 72: 'PR', 78: 'VI'  # Territories: Guam, Puerto Rico, Virgin Islands
}

# Convert float state codes to abbreviations
scoringdf['_STATE'] = scoringdf['_STATE'].map(state_mapping)

#now I want to only keep the states that are also in the daylight states
valid_states = {
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA',
    'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD',
    'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ',
    'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
    'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY',
    'PR'  # Puerto Rico
}

scoringdf = (scoringdf[scoringdf['_STATE'].isin(valid_states)]).dropna()
scoringdf.head()

,_STATE,MENTHLTH,ADDEPEV3
0,AL,88.0,2.0
1,AL,88.0,2.0
2,AL,3.0,2.0
3,AL,88.0,2.0
4,AL,88.0,2.0


In [7]:
scoringdf.describe()

,MENTHLTH,ADDEPEV3
count,412692.000000,412692.000000
mean,58.385554,1.827782
std,37.842242,0.609071
min,1.000000,1.000000
25%,14.000000,2.000000
50%,88.000000,2.000000
75%,88.000000,2.000000
max,99.000000,9.000000


In [8]:
#goal here is to normalize the values and add a Depres Index column

#there are values in the columns that are place holder values that represent something else ie. refused to answer, None, didnt answer.
menthlth_invalid = {77.0, 99.0}
addepev3_invalid = {7.0, 9.0}

#get rid of invalid values for the 2 columns
scoringdf = scoringdf[~scoringdf['MENTHLTH'].isin(menthlth_invalid) & ~scoringdf['ADDEPEV3'].isin(addepev3_invalid)].copy()

# Convert ADDEPEV3 (Depression Diagnosis) into binary format
scoringdf["ADDEPEV3"] = scoringdf["ADDEPEV3"].map({1.0: 1.0, 2.0: 0.0})

# Convert special code 88 for MENTHLTH to reflect zero days
MENTHLTH_map = {1.:1., 2.:2., 3.:3., 4.:4., 5.:5., 6.:6., 7.:7., 8.:8.,
                9.:9., 10.:10., 11.:11., 12.:12., 13.:13., 14.:14., 15.:15.,
                16.:16, 17.:17., 18.:18., 19.:19., 20.:20., 21.:21., 22.:22.,
                23.:23., 24.:24., 25.:25., 26.:26., 27.:27., 28.:28., 29.:29.,
                30.:30., 88:0.}

scoringdf['MENTHLTH'] = scoringdf['MENTHLTH'].map(MENTHLTH_map)

#Verify data
print(f'Unique values for ADDEPEV3: {scoringdf.ADDEPEV3.unique()}')
print(f'Unique values for MENTHLTH: {sorted(scoringdf.MENTHLTH.unique())}')
scoringdf.head()

Unique values for ADDEPEV3: [0. 1.]
Unique values for MENTHLTH: [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0]


,_STATE,MENTHLTH,ADDEPEV3
0,AL,0.0,0.0
1,AL,0.0,0.0
2,AL,3.0,0.0
3,AL,0.0,0.0
4,AL,0.0,0.0


In [9]:
scoringdf.describe()

,MENTHLTH,ADDEPEV3
count,402198.000000,402198.00000
mean,4.365591,0.20695
std,8.374136,0.40512
min,0.000000,0.00000
25%,0.000000,0.00000
50%,0.000000,0.00000
75%,4.000000,0.00000
max,30.000000,1.00000


In [10]:
#then we want to scale the number of bad mental health days between 0 and 1
#so that it is on the same scale as the ADDEPEV3
scaleddf = scoringdf.copy()
scaler1 = MinMaxScaler()
scaleddf['MENTHLTH'] = scaler1.fit_transform(scaleddf['MENTHLTH'].values.reshape(-1, 1))

#now apply the weights that we set for our Depression Index
scaleddf['ADDEPEV3'] = scaleddf['ADDEPEV3'] * 10
scaleddf['MENTHLTH'] = scaleddf['MENTHLTH'] * 8

#make a column for the DI that is both columns added together, on a 0-10 scale.
scaler2 = MinMaxScaler(feature_range=(0,10))
scaleddf['DI'] = scaler2.fit_transform(
    (scaleddf['MENTHLTH'] + scaleddf['ADDEPEV3']).values.reshape(-1, 1)
    )
scaleddf.head()

,_STATE,MENTHLTH,ADDEPEV3,DI
0,AL,0.0,0.0,0.000000
1,AL,0.0,0.0,0.000000
2,AL,0.8,0.0,0.444444
3,AL,0.0,0.0,0.000000
4,AL,0.0,0.0,0.000000


In [11]:
# Read transformed daylight notebook in from disk
daylight_trans = pd.read_csv(PATH + 'daylight_transformed.csv')
daylight_trans.head()

,State,Begin Civil Twilight,Rise,Upper Transit,Set,End Civil Twilight,Daylight Hours,DH,DH_Z-score,DH_min-max
0,AL,2022-12-21 04:15:00,2022-12-21 04:42:00,2022-12-21 09:43:00,2022-12-21 14:44:00,2022-12-21 15:11:00,0 days 10:02:00,10.033333,0.987589,0.782918
1,AK,2022-12-21 08:52:00,2022-12-21 09:45:00,2022-12-21 12:56:00,2022-12-21 16:07:00,2022-12-21 17:00:00,0 days 06:22:00,6.366667,-4.379642,0.000000
2,AZ,2022-12-21 06:01:00,2022-12-21 06:28:00,2022-12-21 11:26:00,2022-12-21 16:25:00,2022-12-21 16:52:00,0 days 09:57:00,9.950000,0.865607,0.765125
3,AR,2022-12-21 04:45:00,2022-12-21 05:13:00,2022-12-21 10:07:00,2022-12-21 15:02:00,2022-12-21 15:30:00,0 days 09:49:00,9.816667,0.670435,0.736655
4,CA,2022-12-21 06:50:00,2022-12-21 07:20:00,2022-12-21 12:04:00,2022-12-21 16:48:00,2022-12-21 17:18:00,0 days 09:28:00,9.466667,0.158108,0.661922


In [12]:
# Merge transformed daylight data and transformed BRFSS data
analysis_df = pd.merge(daylight_trans[['State', 'DH']],
                       scaleddf[['_STATE', 'DI', 'ADDEPEV3', 'MENTHLTH']],
                       left_on='State', right_on='_STATE', how='inner'
                      ).drop('_STATE', axis=1)
analysis_df.head()

,State,DH,DI,ADDEPEV3,MENTHLTH
0,AL,10.033333,0.000000,0.0,0.0
1,AL,10.033333,0.000000,0.0,0.0
2,AL,10.033333,0.444444,0.0,0.8
3,AL,10.033333,0.000000,0.0,0.0
4,AL,10.033333,0.000000,0.0,0.0


In [13]:
# Normalize the 'Daylight Hours' column (Z-score)
analysis_df['DH_Z'] = sps.zscore(analysis_df.DH.values)

# Normalize the Depression Index (Z-score)
analysis_df['DI_Z'] = zscore(analysis_df.DI.values)
analysis_df.describe()

,DH,DI,ADDEPEV3,MENTHLTH,DH_Z,DI_Z
count,402198.000000,402198.000000,402198.000000,402198.000000,4.021980e+05,4.021980e+05
mean,9.312854,1.796478,2.069503,1.164158,4.725433e-15,4.095976e-17
std,0.633300,3.001230,4.051201,2.233103,1.000001e+00,1.000001e+00
min,6.366667,0.000000,0.000000,0.000000,-4.652124e+00,-5.985814e-01
25%,9.000000,0.000000,0.000000,0.000000,-4.940058e-01,-5.985814e-01
50%,9.333333,0.000000,0.000000,0.000000,3.233819e-02,-5.985814e-01
75%,9.566667,2.222222,0.000000,1.066667,4.007790e-01,1.418567e-01
max,11.050000,10.000000,10.000000,8.000000,2.743010e+00,2.733390e+00


Note how the Z-score transforms each data column so that it has zero mean and unit variance.

In [14]:
# Write merged and transformed data frame to disk for collaborative notebooks
analysis_df.to_csv(PATH + 'analysis_2022.csv', index=False)

## Record Dependencies

In [15]:
%load_ext watermark
%watermark
%watermark --iversions

Last updated: 2025-02-17T02:37:10.317533+00:00

Python implementation: CPython
Python version       : 3.10.11
IPython version      : 8.17.2

Compiler    : GCC 11.3.0
OS          : Linux
Release     : 6.5.0-1020-aws
Machine     : x86_64
Processor   : x86_64
CPU cores   : 64
Architecture: 64bit

numpy  : 1.24.3
sklearn: 1.2.2
scipy  : 1.10.1
pandas : 2.0.2

